- 一層目のモデルを作成してoof,testを出力するファイル
- これの次にtubo_stacking.ipynbでsubmitte.csvを作成する。

# Import

In [2]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import mean_squared_error
from fasttext import load_model
from module import features,models #自作モジュールのインポート
from create_dataset import run #大きめな特徴量作るやつ

# Define function

In [3]:
#likesが0より大きいかどうかでStratifiedKFold
def make_cv(n_split,seed):
    fold = StratifiedKFold(n_splits=n_split,shuffle=True,random_state=seed)
    bins = [1 if i > 0 else 0 for i in target.tolist()]
    return list(fold.split(train,bins))

def preprocess(df):
    df[text_col] = df[text_col].fillna('nashi').astype(object)
    fast = load_model('../lid.176.bin')
    df["title_lang_ft"] = df["title"].fillna("").map(
    lambda x: fast.predict(x.replace("\n", ""))[0][0])

    df['make_period'] = df['dating_year_late'] - df['dating_year_early']
    return df

# Config

In [4]:
INPUT_PATH = '../../../data/'
CONFIG_PATH = '../config/'
ENSEMBLE_PATH = '../../../ensemble/'
SEED = 12
N_SPLIT = 5

params_depth3 = json.load(open(CONFIG_PATH + 'config1.json','r'))
params_depth6 = json.load(open(CONFIG_PATH + 'config2.json','r'))
params_depth8 = json.load(open(CONFIG_PATH + 'config3.json','r'))

text_col=['title','description','long_title','more_title','acquisition_credit_line']
cat_col = ['title_lang_ft','principal_maker','principal_or_first_maker','copyright_holder','acquisition_method']

# Load Data

In [5]:
#Load
train = pd.read_csv(INPUT_PATH + 'train.csv')
test = pd.read_csv(INPUT_PATH + 'test.csv')

#まとめて前処理
whole_df = pd.concat([train,test],axis=0).reset_index(drop=True)
whole_df = preprocess(whole_df)
num = len(train)

#分割
train = whole_df.iloc[:num]
test = whole_df.iloc[num:].reset_index(drop=True)

#RMSEで学習できるように対数変換
target = np.log1p(train['likes'])

# Create Dataset

In [5]:
#1時間くらいかかる。大きめな特徴量を別で作成。先に作ってあるので飛ばしてもOK
run.main()

INFO:absl:Using /tmp/tfhub_modules to cache modules.
100%|██████████| 23995/23995 [00:08<00:00, 2846.57it/s]
1/30 * Epoch (train):   0% 0/24 [00:00<?, ?it/s]/home/yuuki/anaconda3/envs/tf/lib/python3.7/site-packages/catalyst/experiments/experiment.py:221: UserWarning:

Attention, there is only one dataloader - train

1/30 * Epoch (train): 100% 24/24 [01:17<00:00,  3.23s/it, loss=0.076]
[2021-03-16 22:48:04,546] 
1/30 * Epoch 1 (_base): lr=0.0010
1/30 * Epoch 1 (train): loss=0.1573
INFO:metrics_logger:
1/30 * Epoch 1 (_base): lr=0.0010
1/30 * Epoch 1 (train): loss=0.1573
2/30 * Epoch (train):  67% 16/24 [01:09<00:15,  1.91s/it, loss=0.090]

# Create Feature

In [6]:
#特徴量の定義
processblock = [
    features.AraiBlock(2),
    *[features.UniversalBlock(col,20) for col in text_col],
    *[features.CountEncodingBlock(col) for col in cat_col],
    features.WrapperBlock(features.identity_func),
    features.WrapperBlock(features.datingdata),
    features.SubtoSizeBlock(),
    features.TechBlock(10),
    features.MaterialBlock(10),
    features.CollectBlock(10),
    features.CountryBlock(),
    features.PersonBlock(),
    features.PaletteBlock(whole_df),
    features.Color2VecBlock(10),
    features.PrincipalMakerBlock(5,'roles'),
    features.PrincipalMakerBlock(5,'qualification'),
    *[features.ColorBlock(num) for num in range(3)],
    *[features.TargetEncodingBlock(name) for name in ['material','technique']],
    *[features.StrCountBlock(col) for col in text_col],
    features.LabelEncodingBlock(['principal_maker','principal_or_first_maker','copyright_holder','acquisition_method','title_lang_ft'],whole_df),
    *[features.BertMultiBlock(col,60) for col in text_col],
    *[features.TfidBlock(whole_df,col,100) for col in text_col]
]

#特徴量作成
train_feat_df = features.to_feature(train,processblock,True)
test_feat_df = features.to_feature(test,processblock,False)

 14%|█▎        | 6/44 [00:26<03:07,  4.93s/it]create<module.features.UniversalBlock object at 0x7f4c7a219b10> 5.175[s]
create<module.features.CountEncodingBlock object at 0x7f4c7a219bd0> 0.002[s]
create<module.features.CountEncodingBlock object at 0x7f4c7a219cd0> 0.003[s]
create<module.features.CountEncodingBlock object at 0x7f4c7a219ed0> 0.002[s]
create<module.features.CountEncodingBlock object at 0x7f4c7a219f10> 0.001[s]
create<module.features.CountEncodingBlock object at 0x7f4c7a219f50> 0.002[s]
create<module.features.WrapperBlock object at 0x7f4c7a219c90> 0.004[s]
 30%|██▉       | 13/44 [00:26<00:35,  1.14s/it]create<module.features.WrapperBlock object at 0x7f4c7a219fd0> 0.162[s]
/workspace/src/tubo/module/features.py:252: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

KeyboardInterrupt: 

# Build Model

In [12]:
#深さ違いのmodelを計９個作成
all_model = [*[models.Lgbm(param) for param in [params_depth3['lgbm_params'],params_depth6['lgbm_params'],params_depth8['lgbm_params']]],
            *[models.Cat(param) for param in [params_depth3['cat_params'],params_depth6['cat_params'],params_depth8['cat_params']]],
            *[models.Xgb(param) for param in [params_depth3['xgb_params'],params_depth6['xgb_params'],params_depth8['xgb_params']]],  
            ]
#stacking用
Ridge = models.Rid()

# Train

In [11]:
cv = make_cv(N_SPLIT,SEED)
depth = [3,6,8]

oof_pred = pd.DataFrame()
test_pred = pd.DataFrame()

#深さごとにモデルをtrain
for i in range(3):
    oof_lgb,test_lgb,__ = all_model[i].cv(target,train_feat_df,test_feat_df,cv)
    oof_cat,test_cat,__ = all_model[i+3].cv(target,train_feat_df,test_feat_df,cv)
    oof_xgb,test_xgb,__ = all_model[i+6].cv(target,train_feat_df,test_feat_df,cv)

    #stacking用dfを作成
    stack_train_df = pd.DataFrame([oof_lgb,oof_cat,oof_xgb]).T
    stack_test_df = pd.DataFrame([test_lgb,test_cat,test_xgb]).T

    #Ridge回帰でstacking
    oof_rid,test_rid,__ = Ridge.cv(target,stack_train_df,stack_test_df,cv)

    _oof_pred = pd.DataFrame([oof_lgb,oof_cat,oof_xgb,oof_rid],
                index=[f'lgb_depth{depth[i]}',
                         f'cat_depth{depth[i]}',
                         f'xgb_depth{depth[i]}',
                         f'rid_depth{depth[i]}']).T
    _test_pred = pd.DataFrame([test_lgb,test_cat,test_xgb,test_rid],
                index=[f'lgb_depth{depth[i]}',
                         f'cat_depth{depth[i]}',
                         f'xgb_depth{depth[i]}',
                         f'rid_depth{depth[i]}']).T

    oof_pred = pd.concat([oof_pred,_oof_pred],axis=1)
    test_pred = pd.concat([test_pred,_test_pred],axis=1)

#oof,testをensemble用フォルダに保存
oof_pred.add_prefix('tubo_').to_csv(ENSEMBLE_PATH+'tubo_train.csv',index=False)
test_predadd_prefix('tubo_').to_csv(ENSEMBLE_PATH+'tubo_test.csv',index=False)

/home/yuuki/anaconda3/envs/tf/lib/python3.7/site-packages/lightgbm/basic.py:1555: UserWarning:

categorical_feature in Dataset is overridden.
New categorical_feature is ['acquisition_method', 'copyright_holder', 'principal_maker', 'principal_or_first_maker', 'title_lang_ft']

/home/yuuki/anaconda3/envs/tf/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning:

Overriding the parameters from Reference Dataset.

/home/yuuki/anaconda3/envs/tf/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning:

categorical_column in param dict is overridden.

fold0:RMSLE 0.9892466232813267
/home/yuuki/anaconda3/envs/tf/lib/python3.7/site-packages/lightgbm/basic.py:1555: UserWarning:

categorical_feature in Dataset is overridden.
New categorical_feature is ['acquisition_method', 'copyright_holder', 'principal_maker', 'principal_or_first_maker', 'title_lang_ft']

/home/yuuki/anaconda3/envs/tf/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning:

Overriding the parameters f